In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/usr0/home/naveenr/projects/food_rescue_feedback')

In [3]:
import numpy as np
import random 
import matplotlib.pyplot as plt
from feedback.fr_feedback import *
from feedback.generate_report import *
import feedback.secret as secret
from feedback.database import open_connection
import pandas as pd

In [12]:
start_date = "2024-05-20"
end_date = "2024-05-26"

## Donors

In [13]:
db_name = "fr_new" 
username = secret.database_username 
password = secret.database_password 
ip_address = "localhost"
port = secret.database_port

In [14]:
connection_dict = open_connection(db_name,username,password,ip_address,port)
connection = connection_dict['connection']
cursor = connection_dict['cursor']

In [15]:
feedbacks = get_feedback_by_date(connection,start_date,end_date)
feedbacks.to_csv('../../results/reports/feedbacks_{}_{}.csv'.format(start_date,end_date), index=False)

/usr0/home/naveenr/projects/food_rescue_feedback/feedback/database.py:59: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)
/usr0/home/naveenr/projects/food_rescue_feedback/feedback/database.py:59: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


In [21]:
annotated_feedback = generate_prompts_and_analyze_feedback(feedbacks)
annotated_feedback.to_csv('../../results/reports/labeled_feedbacks_{}_{}.csv'.format(start_date,end_date), index=False)

On feedback 1 out of 126
Task Description: As an analyst for our food rescue platform, your primary task is to carefully review feedback provided by volunteer drivers, with a specific focus on identifying issues caused by the donors.

Roles Explained:
The Donor - Provides the food.
The Volunteer Driver - Transports the food.
The Recipient - Receives the food.

Volunteer Feedback Analysis:
Volunteers interact with our app to claim tasks, collect food from donors, and deliver it to recipients. They leave comments and rate their experience post-rescue. Your task is to pinpoint and evaluate comments that reflect problems directly caused by the donors, but NOT due to inadequate food or an earlier pickup. 

Guidelines for Analysis:
1. Donor Problem: Assess whether the reported challenges or failures in the food rescue process were caused by communication issues with donors. If the problem is caused by issues with either inadequate food ("missing donation", "not enough food", "no donation", e

KeyError: "['recipient_problem', 'inadequate_food', 'direction_problem', 'earlier_pickup', 'system_problem', 'update_contact'] not in index"

In [9]:
annotated_feedback

,rescue_id,recipient_problem,inadequate_food,donor_problem,direction_problem,earlier_pickup,system_problem,update_contact
0,768853,False,False,False,False,False,False,False
1,773895,False,True,True,False,False,False,False
2,783872,False,True,False,False,False,False,False
3,785523,True,False,True,False,False,False,False
4,770377,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
121,774650,False,False,False,False,False,False,False
122,773872,True,False,True,False,True,False,False
123,767220,False,False,False,False,False,False,False
124,775404,False,True,True,False,False,False,False


In [9]:
# feedbacks = pd.read_csv('../../results/reports/feedbacks_{}_{}.csv'.format(start_date,end_date))
# annotated_feedback = pd.read_csv('../../results/reports/labeled_feedbacks_{}_{}.csv'.format(start_date,end_date))

In [17]:
# Calculate statistics and generate the report
donor_statistics = calculate_donor_statistics(feedbacks, annotated_feedback)
recipient_statistics = calculate_recipient_statistics(feedbacks, annotated_feedback)
report_info = info_update(annotated_feedback)

# Save the report to a CSV file
donor_statistics.to_csv('../../results/reports/report_donor_{}_{}.csv'.format(start_date,end_date), index=False)
recipient_statistics.to_csv('../../results/reports/report_recipient_{}_{}.csv'.format(start_date,end_date), index=False)
report_info.to_csv('../../results/reports/report_info_{}_{}.csv'.format(start_date,end_date), index=False)